In [5]:
bugs = User.objects.filter(username__startswith='(')
user = bugs[0]

In [ ]:
from django.core.exceptions import ObjectDoesNotExist
from utils.merge_model_objects import merge_instances
from django.contrib.auth import get_user_model
from django.contrib.auth.models import Group

User = get_user_model()
MANAGEMENT = 'mellomledere'
STAFF = 'redaksjon'

def add_contributor_to_groups(contributor):
    management, __ = Group.objects.get_or_create(name=MANAGEMENT)
    staff, __ = Group.objects.get_or_create(name=STAFF)
    active_stints = contributor.stint_set

def connect_contributor_to_user(contributor, create=False):
    """Connect contributor to user"""
    if contributor.user:
        return contributor.user
    user = contributor.get_user()
    if user:
        try:
            person = user.contributor
        except ObjectDoesNotExist:
            contributor.user = user
            contributor.save()
        else:
            merge_instances(contributor, person)
    elif create:
        if not contributor.email:
            contributor.email = f'{contributor.display_name}@universitas.no'.lower().replace(' ', '')
            
        username=contributor.email.split('@')[0].replace('.', '').lower()
        if User.objects.filter(username=username):
            username=contributor.email
        
        user = User.objects.create_user(
            username=username,
            email=contributor.email,
            first_name=contributor.first_name,
            last_name=contributor.last_name,
            password=None,
            is_active=True,
        )[0]
        contributor.user = user
        contributor.save()
    return user


for contributor in Contributor.objects.all():
    if contributor.stint_set.active():
        user = connect_contributor_to_user(contributor, create=True)
        print(f'{str(contributor):<30} {user}')

In [ ]:
from django.core.exceptions import ObjectDoesNotExist
from utils.merge_model_objects import merge_instances
from django.contrib.auth import get_user_model
from django.contrib.auth.models import Group

MANAGEMENT = 'mellomledere'
STAFF = 'redaksjon'


def add_contributor_to_groups(contributor):
    management, __ = Group.objects.get_or_create(name=MANAGEMENT)
    staff, __ = Group.objects.get_or_create(name=STAFF)
    active_stints = contributor.stint_set


def connect_contributor_to_user(contributor, create=False):
    """Connect contributor to user"""
    if contributor.user:
        return contributor.user
    user = contributor.get_user()
    if user:
        try:
            person = user.contributor
        except ObjectDoesNotExist:
            person.user = user
            person.save()
        else:
            merge_instances(contributor, person)
    elif create:
        user = get_user_model().create(
            username=contributor.email.split('@')[0].replace('.', '').lower(),
            email=contributor.email,
            first_name=contributor.first_name,
            last_name=contributor.last_name,
            active=True,
        )

    return user


In [ ]:
from django.utils import timezone
now = timezone.now()
users = User.objects.filter(is_active=True).exclude(username__in=['haakenlid', 'api'])
superusers = users.filter(is_superuser=True)
mellomledere = Group.objects.get(name='mellomledere')
redaksjon = Group.objects.get(name='redaksjon')
for user in users: 
    print(f'{str(user):<20} staff:{user.is_staff} super:{user.is_superuser}')
    user.is_superuser = False
    user.groups.add(mellomledere, redaksjon)
    user.user_permissions.clear()
    user.save()
    if not user.last_login or now - user.last_login > timezone.timedelta(days=150):
        print(user, user.last_login)
        user.is_active = False
        user.save()
        

In [ ]:
User.objects.filter(is_active=False).update(is_staff=False, is_superuser=False)

In [ ]:
active_stints = Stint.objects.filter(end_date__gt='2017-07-01')
active_stints.update(end_date='2017-12-31')
for st in active_stints:
    print(f'{str(st.contributor):<30} {str(st.position):<20} {st.start_date} {st.end_date}')

In [ ]:
pos = Position.objects.first()

In [ ]:
for pos in Position.objects.all():
    pos.groups.add(redaksjon)
    if 'redaktør' in pos.title or 'sjef' in pos.title or 'leder' in pos.title:
        pos.groups.add(mellomledere)
        pos.groups.remove(redaksjon)
    print(pos, pos.groups.all())
        

In [ ]:
folk ='''
Annonseansvarlig	Geir Dorp	916 64 496	geirdo@universitas.no
Ansvarlig redaktør	Birk Tjeldflaat Helle	988 51 171	birktjeldflaathelle@gmail.com
Daglig leder	Joakim Stene Preston	955 29 599	j.s.preston@universitas.no
Nyhetsredaktør	Mads Randen	454 00 166	madsranden95@gmail.com
Anmelderredaktør	Mathias Gravdehaug	415 97 277	mathias.gravde@gmail.com
Debattredaktør	Runa Fjellanger	901 59 831	runafjellanger@gmail.com
Deskjournalist	Nora Nygaard	952 82 722	noranygaard@hotmail.com
Desksjef	Lise Blekastad	974 75 088	liseblekastad@gmail.com
Fotosjef	Erlend Daae	900 79 076	erlend.daae@gmail.com
Gravejournalist	Adrian Simen Holm	450 15 880	adrian.simen.holm@gmail.com
Gravejournalist	Gard Oterholm	938 97 476	gard.oterholm@outlook.com
Kulturredaktør	Ella Regine Stokstad	470 21 014	ella.regine@mac.com
Magasinfotosjef	Kristina Elisabet Kvammen	962 23 623	kristinaelisabet@gmail.com
Magasinredaktør	Signe Rosenlund-Hauglid	924 49 934	s.rosenlund.hauglid@gmail.com
Nettredaktør	Markus Slettholm	993 99 054	slettholm@gmail.com
Utenriksredaktør	Emilie Louise Solberg	902 16 573	emilie.solberg93@gmail.com
Fotograf	Alf Simensen	979 74 135	alf-ags@hotmail.com
Fotograf	Amanda O. Berg	410 45 515	amanda.o.berg@gmail.com
Fotograf	Christian Breidlid	974 28 387	christianbreidlid@gmail.com
Fotograf	Dorthe Karlsen	988 58 515	dorthekarlsen@outlook.com
Fotograf	Eskil Wie	954 25 649	eskilwie@gmail.com
Fotograf	Hanna Kristin Hjardar	412 13 731	hanna@hjardar.no
Fotograf	Hanne Marie Solbø	979 68 656	hannesolbo@yahoo.no
Fotograf	Håkon Benjaminsen	943 74 923	hakonben@gmail.com
Fotograf	Matthis Kleeb Solheim	482 99 849	matthiskleeb@hotmail.com
Fotograf	Melisa Fajkovic	902 36 225	melisafajkovic@gmail.com
Fotograf	Odin Drønen	958 28 296	odindro@gmail.com
Fotograf	Odin Jæger	473 98 053	odinja@gmail.com
Fotograf	Siri Øverland Eriksen	936 31 040	sirioeriksen@gmail.com
Fotograf	Sjur Gausemel Stølen 	477 53 805	sjur@sjurgs.com
Fotograf	Vilde Iren Borse	920 68 126	vildeborse@hotmail.com
Fotograf	Xueqi Pang	936 80 955	xueqipang@hotmail.no
Fotograf	Øyvind Aukrust 	400 47 513	oyvind.aukrust@gmail.com
Journalist	Torgeir Mortensen	454 72 320	torgeirm3b@gmail.com
Journalist	Morten Oftedal Schwencke	977 73 328	morten.schwencke@gmail.com
Journalist	Knut Arne Oseid	900 90 332	oseid.knut@gmail.com
Journalist	Philip André Johannseborg	993 97 202	philip.johannesborg@gmail.com
Journalist	Kaja Storrøsten	482 56 156	kajastorrosten@gmail.com
Journalist	Sondre Moen Myhre	907 46 802	sondre.myhre1@gmail.com
Journalist	Bjørnar Konglevoll Bekkevard	950 03 236	bjoernar.k.bekkevard@gmail.com
Journalist	Carina Hunshamar	920 88 193	carinahunshamar@gmail.com
Journalist	Emili Knutson	466 52 689	emili.knutson@gmail.com
Journalist	Hanad Mohamed Ali	413 80 351	fl8hanad@hotmail.no
Journalist	Hanna Skotheim	917 84 820	skotheimhanna@gmail.com
Journalist	Hannah Berg	918 48 394	hannah.berg@live.no
Journalist	Håvard Røsæg	954 59 260	haavard24@live.com
Journalist	Ida Lyngstad Wernø	958 67 095	ida.werno@gmail.com
Journalist	Ida Wammer	940 33 277	ida.wammer@gmail.com
Journalist	Ingeborg Misje	954 10 414	fukausen@hotmail.com
Journalist	Julie Bjander	404 62 489	julie.bjander@hotmail.com
Journalist	Julie Løvseth	907 46 147	juliebrundtland@hotmail.com
Journalist	Kenneth Haug	950 16 594	kenneth.kandolf.haug@gmail.com
Journalist	Knut Ward Heimdal	980 99 488	knut.heimdal@gmail.com
Journalist	Kristina Holt	986 23 799	kristina.holt@icloud.com
Journalist	Lina Christensen	970 97 251	linachristens@gmail.com
Journalist	Louisa Boulaziz	400 94 261	louisaboulaziz@gmail.com
Journalist	Magnus Godvik Ekeland	924 69 786	magnusekeland@hotmail.com
Journalist	Mari Mjaaland	473 47 210	mari.mjaaland@gmail.com
Journalist	Heidi Bang		bang.heidi@gmail.com
Journalist	Rachel Antonsen		racheleantonsen@gmail.com
Journalist	Henrik Giæver		henrikgiaever@hotmail.com
Journalist	Benedicte Tobiassen		benedictetobiassen@gmail.com
Journalist	Stine Gudmundsen Bergo		stine_gb_@hotmail.com
Journalist	Arvid Folke Järnbert		folkejarnbert@gmail.com
Journalist	Vebjørn Wold		vebjor2910@gmail.com
Journalist	Maria Pettrém		mariapettrem@gmail.com
Journalist	Ingrid Ekeberg		ekeberg.ingrid@gmail.com
Uttegner	Axel Hodnefjeld	450 90 050	axel.hodnefjeld@gmail.com
Uttegner	Adrian Nielsen	454 46 444	adrian.nielsen@gmail.com
Uttegner	Anne Garvoll	476 68 428	annegarvoll@hotmail.com
Uttegner	Kari Eiring	979 81 175	karieiring@gmail.com
Uttegner	Mats Johannesen	934 06 680	mats@matsj.net
'''


In [ ]:
from collections import namedtuple
Person = namedtuple('Person', 'title, name, tlf, email')
lista = [Person(*line.split('\t')) for line in folk.strip().splitlines()]
lista.sort(key=lambda p: p.name)
 
for person in lista:
    first, *m, last = person.name.split(' ')
    username = person.email.split('@')[0].replace('.', '').lower()
    user = User.objects.filter(first_name=first, last_name=last).first()
    if not user:
        user = User.objects.filter(email=person.email).first()
    if not user:
        user = User.objects.filter(username=username).first()
    if user:
        if not user.email: 
            user.email = person.email
        if person.title in ['Fotograf', 'Uttegner', 'Journalist']:
            user.groups.add(redaksjon) 
        else:
            print(person.name, person.title)
            user.groups.add(mellomledere)
        user.save()
    else:
        print(person.name)
        User.objects.create_user(
            first_name=first,
            last_name=last,
            email=person.email,
            username=username,
            password='EhmfnptU',
            is_active=True,
        )

In [ ]:
User.objects.filter(username='oyvindaukrust').first()

In [ ]:
import json
import pprint
jsondump = r'''
{"count":16,"next":null,"previous":null,"results":[{"id":62586,"url":"http://universitas.no/api/stories/62586/","public_url":"http://universitas.no/nyheter/62586/","edit_url":"http://universitas.no/admin/stories/story/62586/change/","modified":"2017-09-03T17:52:48.007466Z","created":"2017-09-03T17:38:25.595855Z","working_title":"Nyhet 1","publication_status":100,"bodytext_markup":"1ER-SAKER ER SAKENE SOM PRIORITERES PÅ EN DOBBELTSIDE. DE ER ALLTID AVHENGIGE AV AT BILDET, ELLER ILLUSTRASJONEN ER BRA.\nSIDE 4-5: UTEN ANDRE SAKER: 4000 TEGN I HOVEDSAK, CA 1900 I UNDERSAK. MED ÉNSPALTER: 4000 – 4200. MED 2-SPALTER: 3800 – 4000. MED 3-SPALTER: CA 3800 TEGN. 8-SPALTER MED HEMS: CA 4300 TEGN. DOBBELTSIDE: MAKS 4200 TEGN I HOVEDSAK, CA 1900 I UNDERSAK. NB: MAKS 4200 TEGN I HOVEDSAKEN. BRUK UNDERSAK OM SAKEN SKAL VÆRE LENGRE ENN DETTE (MAKS 1900 TEGN).\n\n@stikktit:(PRØV Å UNNGÅ STIKKTITLER – STIKKTITLER ER SOM STØTTEHJUL!)\n\n@tit: \n\n@ing: \n\n@li:INGRESS1:\n\n@li:INGRESS2:\n@bt:\n\n@bt:\n\n@tema:\n\n@bl:\n\n@txt:\n\n@mt:\n\n@txt:\n\n@mt:\n\n@txt:\n\n@sitat:\n@sitatbyline:\n\n@fakta:\n\n@tit:(undersak)\n\n@txt:(IKKE INGRESS I UNDERSAK)\n","story_type":2,"story_type_name":"Nyhet","byline_set":[]},{"id":62596,"url":"http://universitas.no/api/stories/62596/","public_url":"http://universitas.no/kultur/62596/","edit_url":"http://universitas.no/admin/stories/story/62596/change/","modified":"2017-09-03T17:51:59.431198Z","created":"2017-09-03T17:51:03.661346Z","working_title":"Kultur 3","publication_status":100,"bodytext_markup":"3ERSAKENE PÅ KULTUR ER SAKENE SOM TAR 50 % ELLER MINDRE AV PLASSEN PÅ EN SIDE, ELLER DOBBELTSIDE.\n2-SPALTER MED HEMS: CA 2800 TEGN. 3-SPALTER MED HEMS: CA 3500 TEGN (MÅ HA FOTOLØSNING). ÉNSPALTER UTEN HEMS: MAKS 2000 TEGN. HALVSIDE: 2600 – 2800 TEGN (MED STÅENDE BILDE). \n\n@tit:\n\n@ing:\n\n@bt:(SKRIV TI ORD TIL HVERT AV FRIMERKEBILDENE)\n\n@bt:\n\n@bl:\n\n@txt:Temaord:\n\n@mt:\n\n@txt:\n\n@mt:\n\n@txt:\n\n@sitat:\n\n@sitatbyline:\n\n@txt:xx@universitas.no\n\n(LEGG ALLTID INN ETT ELLER TO FRIMERKEBILDER / TRYNEBILDER AV DE SOM UTTALER SEG I SAKEN).\n\n(NB! SELV OM DET KOMMER «WARNING» I PRODSYS NÅR @ STÅR I TEKSTEN, SKAL IKKE EPOSTADRESSE FJERNES. DET ER BARE «ERROR» SOM IKKE LAR DEG LAGRE)","story_type":10,"story_type_name":"Kultur","byline_set":[]},{"id":62595,"url":"http://universitas.no/api/stories/62595/","public_url":"http://universitas.no/kultur/62595/","edit_url":"http://universitas.no/admin/stories/story/62595/change/","modified":"2017-09-03T17:51:49.222941Z","created":"2017-09-03T17:50:57.659416Z","working_title":"Kultur 2","publication_status":100,"bodytext_markup":"2ERSAKENE PÅ KULTUR ER SAKENE SOM TAR MEST PLASS PÅ EN SIDE, ELLER TAT HELE SIDEN ALENE. SOM ALLTID: VELDIG AVHENGIG AV BRA FOTOLØSNING, KUL ILLUSTRASJON ELLER GRAFIKK.\nMED HEMS ELLER FEM PÅ PLASSEN: 2800 – 3000 TEGN. MED ÉNSPALTER: MAKS 3400. HELE SIDEN: MAKS 3800 TEGN.\n\n@tit:\n\n@ing:\n\n@bt:\n\n@bt:\n\n@bl:\n\n@txt:Temaord:\n\n@mt:\n\n@txt:\n\n@mt:\n\n@txt:\n\n@sitat:\n\n@sitatbyline:\n\n@fakta:\n\n@txt:xx@universitas.no\n\n(NB! SELV OM DET KOMMER «WARNING» I PRODSYS NÅR @ STÅR I TEKSTEN, SKAL IKKE EPOSTADRESSE FJERNES. DET ER BARE «ERROR» SOM IKKE LAR DEG LAGRE)\n","story_type":10,"story_type_name":"Kultur","byline_set":[]},{"id":62585,"url":"http://universitas.no/api/stories/62585/","public_url":"http://universitas.no/kultur/62585/","edit_url":"http://universitas.no/admin/stories/story/62585/change/","modified":"2017-09-03T17:51:30.259825Z","created":"2017-09-03T17:38:21.380213Z","working_title":"Kultur 1","publication_status":100,"bodytext_markup":"1ERSAKENE TAR MER ENN 50 % PÅ EN DOBBELTSIDE. PÅ KULTUR KREVER DE MEGET GODE BILDER OG TEKSTLENGDE VARIERER VELDIG I FORHOLD TIL HVOR GODE BILDENE, GRAFIKKEN, ELLER ILLUSTRASJONEN ER.\nINNGANGSSIDE / DOBBELTSIDE MED HEMS: 7-SPALTER: 4500 – 4800 TEGN. 8-SPALTER: 4500 – 5000 TEGN. DOBBELTSIDE: 5000 – 6500 (VELDIG BILDEAVHENGIG).\n\n@tit:\n\n@ing:\n\n@bt:\n\n@bt:\n\n@bl:\n\n@txt:Temaord:\n\n@mt:\n\n@txt:\n\n@mt:\n\n@txt:\n\n@sitat:\n\n@sitatbyline:\n\n@fakta:\n\n@txt:xx@universitas.no\n\n(NB! SELV OM DET KOMMER «WARNING» I PRODSYS NÅR @ STÅR I TEKSTEN, SKAL IKKE EPOSTADRESSE FJERNES. DET ER BARE «ERROR» SOM IKKE LAR DEG LAGRE)","story_type":10,"story_type_name":"Kultur","byline_set":[]},{"id":62593,"url":"http://universitas.no/api/stories/62593/","public_url":"http://universitas.no/nyheter/62593/","edit_url":"http://universitas.no/admin/stories/story/62593/change/","modified":"2017-09-03T17:50:46.527147Z","created":"2017-09-03T17:49:28.622573Z","working_title":"Nyhet 2","publication_status":100,"bodytext_markup":"2ER-SAKER ER SAKENE SOM TAR HELE ELLER MER ENN 50 % AV EN ENKELTSIDE. BILDE ELLER ILLUSTRASJON MÅ VÆRE TIPP TOPP.\nHELE SIDEN: 3500 – 4000 TEGN. MED ÉNSPALTER, FEM PÅ PLASSEN, ELLER HEMS: 3000 – 3500 TEGN.\n\n@stikktit:(PRØV Å UNNGÅ STIKKTITLER – STIKKTITLER ER SOM STØTTEHJUL!)\n\n@tit:\n\n@ing: \n @bt:\n\n@bt:\n\n@tema:\n\n@bl:\n\n@txt:\n\n@mt:\n\n@txt:\n\n@mt:\n\n@txt:\n @sitat:\n\n@sitatbyline:\n\n@fakta:\n @txt:xx@universitas.no\n\n (NB! SELV OM DET KOMMER «WARNING» I PRODSYS NÅR @ STÅR I TEKSTEN, SKAL IKKE EPOSTADRESSE FJERNES. DET ER BARE «ERROR» SOM IKKE LAR DEG LAGRE)\n\n","story_type":2,"story_type_name":"Nyhet","byline_set":[]},{"id":62594,"url":"http://universitas.no/api/stories/62594/","public_url":"http://universitas.no/nyheter/62594/","edit_url":"http://universitas.no/admin/stories/story/62594/change/","modified":"2017-09-03T17:50:42.830916Z","created":"2017-09-03T17:49:55.800802Z","working_title":"Nyhet 3","publication_status":100,"bodytext_markup":"3ERSAKER ER SAKENE SOM TAR MINDRE ENN HALVPARTEN AV SIDEN, ELLER DELER SIDE MED 50 % ELLER MER REKLAME.\n\nÉNSPALTER: 2000 TEGN (KUN SMÅ FRIMERKEBILDER). TOSPALTER: 3200 TEGN (LIGGENDE BILDE, ELLER FRIMERKEBILDE MED UTHEVET SITAT). HALVSIDER: 2600 – 2800 TEGN (BØR HA STÅENDE BILDE).\n\n@tit:KORT TITTEL \n\n@ing:\n\n@bt:(HUSK KJAPP BILDETEKST TIL FRIMERKEBILDENE OGSÅ. MAKS TI ORD)\n @temaord:\n\n@bl:\n\n@txt:\n\n@mt:\n\n@txt:\n\n@mt:\n @sitat:\n\n@sitatbyline:\n @txt:xx@universitas.no\n\nHUSK Å FIKSE FRIMERKEBILDER / PRESSEBILDER AV DE SOM UTTALER SEG I SAKEN OG LEGG DE I NYHETSMAPPA PÅ DESKSERVERN. SKRIV HVEM DET ER I BILDETEKSTEN.\n\n(NB! SELV OM DET KOMMER «WARNING» I PRODSYS NÅR @ STÅR I TEKSTEN, SKAL IKKE EPOSTADRESSE FJERNES. DET ER BARE «ERROR» SOM IKKE LAR DEG LAGRE)\n","story_type":2,"story_type_name":"Nyhet","byline_set":[]},{"id":62592,"url":"http://universitas.no/api/stories/62592/","public_url":"http://universitas.no/side-2/62592/","edit_url":"http://universitas.no/admin/stories/story/62592/change/","modified":"2017-09-03T17:48:38.644466Z","created":"2017-09-03T17:48:23.595836Z","working_title":"Kommentar","publication_status":100,"bodytext_markup":"LENGDE: 3100 – 3200 TEGN\n\n@stikktit:\n\n@tit:\n\n@txt:\n\n@sitat:\n\n@bt:Illustrasjon:\n\n@bl:xx, journalist i Universitas \n\n@txt:xx@universitas.no\n","story_type":7,"story_type_name":"Kommentar","byline_set":[]},{"id":62591,"url":"http://universitas.no/api/stories/62591/","public_url":"http://universitas.no/feature/62591/","edit_url":"http://universitas.no/admin/stories/story/62591/change/","modified":"2017-09-03T17:48:17.007433Z","created":"2017-09-03T17:47:11.165539Z","working_title":"Min studietid","publication_status":100,"bodytext_markup":"FRA 2800 – 2900 TEGN\n\n@tit:\n\n@ing: [HUSK TABULATOR ]\nHVEM:\tnavn\nSTUDERTE:\tstudium\nNÅR:\t18XX-17XX\nAKTUELL MED:\tnoe greier\n\n@txt:\n\n@sitat:\n\n@bl:\n\n@txt:\n\n@bt:Foto: [kun dersom fotograf ikke er en av våre]\n\n","story_type":14,"story_type_name":"Min studietid","byline_set":[]},{"id":62584,"url":"http://universitas.no/api/stories/62584/","public_url":"http://universitas.no/anmeldelser/62584/","edit_url":"http://universitas.no/admin/stories/story/62584/change/","modified":"2017-09-03T17:47:33.106138Z","created":"2017-09-03T17:38:17.694753Z","working_title":"Anmeldelse","publication_status":100,"bodytext_markup":"@li:Verkstittel (TITTEL PÅ VERKET ELLER NAVNET PÅ ARTIST, HVIS DET ER KONSERT)\n\n@li:Av: xxx (BRUK TO ELLER TRE FAKTA OM DET SOM ANMELDES)\nForlag: xxx\nPlateselskap: xxx\nScene: xxx\nRegi: xxx\nMed: xxx\nTid: xxx\n\n@tit:Tittel på sak (MAKS 20 TEGN MED MELLOMROM) \n\n@tema:Sjanger (SKAL VÆRE GENERISK: PLATE, KONSERT, TIDSSKRIFT, UTSTILLING, FILM, ROMAN, SAKPROSA, NOVELLESAMLING, ETC.)\n\n@ing:Ingress (BARE PÅ STOR ANMELDELSE.70 TEGN PÅ EN LINJE, 140 PÅ TO)\n\n@txt:Brødtekst ( LITEN ANMELDELSE 1500 TEGN, STOR ANMELDELSE: 2200 TEGN)\n\n@bt:bildetekst (BARE PÅ STOR ANMELDELSE)\n\n@bl:Byline\n\n–\n\nOBS: TITTEL PÅ PLATER, BØKER OG FILMER SKAL SKRIVES I KURSIV\nTITTEL PÅ SANGER SKAL SKRIVES I «HERMETEGN» I BRØDTEKSTEN.\nI FAKTADELEN SKRIVES ALT RETT FRAM UTEN HERMETEGN\n","story_type":8,"story_type_name":"Anmeldelse","byline_set":[]},{"id":62590,"url":"http://universitas.no/api/stories/62590/","public_url":"http://universitas.no/annet/62590/","edit_url":"http://universitas.no/admin/stories/story/62590/change/","modified":"2017-09-03T17:47:08.130956Z","created":"2017-09-03T17:46:52.396407Z","working_title":"Quiz","publication_status":100,"bodytext_markup":"HUSK Å LEGGE BYLINEBILDE I DEBATTMAPPEN PÅ DESKEN\n\n@tit:\n\n@bl:\n\n@tema:\n\n@txt:\n","story_type":34,"story_type_name":"Quiz","byline_set":[]},{"id":62582,"url":"http://universitas.no/api/stories/62582/","public_url":"http://universitas.no/side-2/62582/","edit_url":"http://universitas.no/admin/stories/story/62582/change/","modified":"2017-09-03T17:45:48.904406Z","created":"2017-09-03T17:30:47.884184Z","working_title":"Kommentar","publication_status":100,"bodytext_markup":"LENGDE: 3100 – 3200 TEGN\n\n@stikktit:\n@tit:\n@txt:\n@sitat:\n@bt:Illustrasjon:\n@bl:xx, journalist i Universitas \n","story_type":7,"story_type_name":"Kommentar","byline_set":[]},{"id":62581,"url":"http://universitas.no/api/stories/62581/","public_url":"http://universitas.no/baksiden/62581/","edit_url":"http://universitas.no/admin/stories/story/62581/change/","modified":"2017-09-03T17:45:37.506553Z","created":"2017-09-03T17:29:50.073865Z","working_title":"Vi Spør","publication_status":100,"bodytext_markup":"LENGDE: 3100 – 3200 TEGN\n\n@stikktit:\n@tit:\n@spm: – spørsmål\n@txt: – svar\n\n@spm: – spørsmål\n@txt: – svar\n\n@spm: – spørsmål\n@txt: – svar\n\n@spm: – spørsmål\n@txt: – svar\n\n@sitat:\n@bt:Illustrasjon:\n@bl:xx","story_type":1,"story_type_name":"Vi spør","byline_set":[]},{"id":62589,"url":"http://universitas.no/api/stories/62589/","public_url":"http://universitas.no/side-2/62589/","edit_url":"http://universitas.no/admin/stories/story/62589/change/","modified":"2017-09-03T17:44:29.204893Z","created":"2017-09-03T17:44:15.400334Z","working_title":"Leder","publication_status":100,"bodytext_markup":"2200 TEGN MED ÉN LEDER. CA 2100 TIL SAMMEN MED TO ELLER MED SITAT.\n\n@tit:\n@txt:\n@sitat:\n\n@tit:\n@txt:\n\n","story_type":11,"story_type_name":"Leder","byline_set":[]},{"id":62588,"url":"http://universitas.no/api/stories/62588/","public_url":"http://universitas.no/anmeldelser/62588/","edit_url":"http://universitas.no/admin/stories/story/62588/change/","modified":"2017-09-03T17:42:30.328837Z","created":"2017-09-03T17:42:00.244579Z","working_title":"Ukas Advarsel / Anbefaling","publication_status":100,"bodytext_markup":"@tit: Tittel (OPPTIL 40 TEGN)\n\n@li: Hva er det? (ETT ORD)\n\n@li: Hvem: xxx\nHvor: xxx\nNår:  xxx (fak 1 og fak 2 SKAL VÆRE MAX 70 TEGN TIL SAMMEN)\n\n@txt: CIRKA 630 TEGN\n\n@bl:xx, journalist i Universitas\n\n–  \nHUSK Å TA BYLINEBILDE OG HUSK Å IKKE VÆRE FOR GENERELL.\n\nLAGRES SOM ARITKKELTYPE «TO UTVALGTE», HA MED ORDET «ADVARSEL» ELLER «ANBEFALING» I ARBEIDSTITTELEN.","story_type":39,"story_type_name":"Ukas Advarsel","byline_set":[]},{"id":62583,"url":"http://universitas.no/api/stories/62583/","public_url":"http://universitas.no/baksiden/62583/","edit_url":"http://universitas.no/admin/stories/story/62583/change/","modified":"2017-09-03T17:41:47.429176Z","created":"2017-09-03T17:38:11.695631Z","working_title":"Ad Notam","publication_status":100,"bodytext_markup":"@tit: \n\n@mt:\n\n@txt:\n\n@mt:\n\n@txt:\n\n@mt:\n\n@txt:\n\n@mt:\n\n@txt:\n\n@mt:\n\n@txt:\n\n@bt:\n\n@bt:\n\n–\nETT ELLER TO BILDER I HØYDEFORMAT.\nMED ETT BILDE: 2500 TEGN (inkl. Mellomrom)\nMED TO BILDER: 2400 TEGN (inkl. Mellomrom)","story_type":15,"story_type_name":"Ad Notam","byline_set":[]},{"id":62587,"url":"http://universitas.no/api/stories/62587/","public_url":"http://universitas.no/nyheter/62587/","edit_url":"http://universitas.no/admin/stories/story/62587/change/","modified":"2017-09-03T17:40:35.900638Z","created":"2017-09-03T17:38:30.167695Z","working_title":"Fem på plassen","publication_status":100,"bodytext_markup":"@ing:Spørsmål (IKKE LEDENDE!!)\n\n@mt:Navn (alder)\n@txt:skole, fag\n@txt:MAX 150 TEGN\n\n@mt:Navn (alder)\n@txt:skole, fag\n@txt:MAX 150 TEGN\n\n@mt:Navn (alder)\n@txt:skole, fag\n@txt:MAX 150 TEGN\n\n@mt:Navn (alder)\n@txt:skole, fag\n@txt:MAX 150 TEGN\n\n@mt:Navn (alder)\n@txt:skole, fag\n@txt:MAX 150 TEGN\n\n@mt:Navn (alder)\n@txt:skole, fag\n@txt:MAX 150 TEGN\n\n","story_type":40,"story_type_name":"Fem på plassen","byline_set":[]}]}
'''
print(jsondump[500:600])
data = json.loads(jsondump)
stories = data['results'])

for 

In [ ]:
new = User.objects.order_by('-pk').filter(is_active=True, is_staff=False)
for p in new:
    print(f'{p.get_full_name():<25}\t{p.username:<20}\t{p.email:<20}')

In [ ]:
allesammen = '''
Adrian Nielsen	454 46 444	adrian.nielsen@gmail.com
Adrian Simen Holm	450 15 880	adrian.simen.holm@gmail.com
Agnes Klem	452 20 554	agnesklem@me.com
Agnes Østengen	994 74 163	agnesoestengen@gmail.com
Agnete Klevstrand	905 55 961	agnete.klevstrand@gmail.com
Aili Røtterud Løchen	957 44 783	aili.rotterud.lochen@gmail.com
Aksel Schreiner Brakestad	971 14 247	akselbrakestad@gmail.com
Aleksander Myklebust	958 57 789	post@almyfoto.com
Alf Simensen	979 74 135	alf-ags@hotmail.com
Amanda Borch Pacheco	906 70 373	amanda.borch.pacheco@gmail.com
Amanda O. Berg	410 45 515	amanda.o.berg@gmail.com
Anders Ballangrud	482 89 939	anba0503@gmail.com
Anders Bjørkheim	476 19 161	bremstol@gmail.com
Anders Fjellberg	993 67 068	a_fjellberg@hotmail.com
Anders Sondrup	971 08 086	an.sond@hotmail.com
Anders Veberg	906 92 963	andveb@gmail.com
Andreas Løhren	951 79 460	andreas.lohren84@gmail.com
Ane Hem	450 51 039	anesteinsholthem@hotmail.com
Anja Elen Eikenes	977 61 947	anjaee@gmail.com
Anja Aaheim Naper	415 51 580	anjanaper@gmail.com
Anne Garvoll	476 68 428	annegarvoll@hotmail.com
Anne Høyer	977 38 036	annehoy@universitas.no
Arvid Folke Järnbert		folkejarnbert@gmail.com
Astrid Karstensen	477 52 707	a.j.karstensen@universitas.no
Astrid Linnea Hovde	452 54 854	astridlinnea@gmail.com
Aurora Hannisdal	415 17 238	auroraah@universitas.no
Axel Hodnefjeld	450 90 050	axel.hodnefjeld@gmail.com
Axel Munthe-Kaas Hærland	901 04 411	amhaerla@student.uio.no
Axel Nygaard	906 59 740	axel.nygaard@gmail.com
Bendik Baksaas	909 27 725	bendik.baksaas@gmail.com
Bendik Løve	917 15 773	bendikl@universitas.no
Benedicte Elisabeth Bjerknes	901 25 184	benedictebjerknes@gmail.com
Benedicte H. Tandsæther-Andersen	478 95 505	benedicte.tandsaether.andersen@gmail.com
Benedicte Tobiassen	997 74 772	benedictetobiassen@gmail.com
Benjamin Aarø	984 80 281	benjamincaaro@gmail.com
Birgitte Hvidsten	928 24 241	birgitte.hvidsten@gmail.com
Birk Tjeldflaat Helle	988 51 171	birktjeldflaathelle@gmail.com
Birte Nystad Magnussen	970 41 577	birteamanda@yahoo.no
Bjørg Sabrin Timraz	483 69 936	bjorgsti@hotmail.no
Bjørnar Konglevoll Bekkevard	950 03 236	bjoernar.k.bekkevard@gmail.com
Brita Melissa Botnen Søreng	917 67 429	brita.soreng@gmail.com
Carina Hunshamar	920 88 193	carinahunshamar@gmail.com
Caroline Ulvin Johansson	913 95 758	caroline.u.johansson@gmail.com
Cecilia Johansen Eriksen	936 31 807	cecilia.johansen92@gmail.com
Christian Breidlid	974 28 387	christianbreidlid@gmail.com
Christian Lycke	922 07 776	christianlycke@gmail.com
Dag Grytli	458 06 409	dagmgr@universitas.no
Dennis	980 25 202	universitas@universitas.no
Dorthe Karlsen	988 58 515	dorthekarlsen@outlook.com
Dziugas Baltrunas	450 40 316	ilammi@hotmail.com
Eir Torvik	413 67 488	eir.torvik@universitas.no
Eirik Elvevold	480 98 064	eirik.b.elvevold@gmail.com
Eirik Omvik	924 93 243	eomvik@gmail.com
Eivind Eide Skaufjord	990 45 537	skaufjord@gmail.com
Eline Anker	957 59 736	elineanker@gmail.com
Ella Regine Stokstad	470 21 014	ella.regine@mac.com
Emili Knutson	466 52 689	emili.knutson@gmail.com
Emilie Louise Solberg	902 16 573	emilie.solberg93@gmail.com
Ena Kreso	992 23 520	enakreso@gmail.com
Endre Stangeby	415 43 840	endrst@universitas.no
Erika Ribu	918 17 724	erikaribu@gmail.com
Erlend Daae	900 79 076	erlend.daae@gmail.com
Eskil Wie	954 25 649	eskilwie@gmail.com
Evelyn Andora Pecori	480 32 049	evelynpecori@gmail.com
Filip Roshauw	452 27 451	filipr@universitas.no
Fredrik Scholze	480 68 710	ezlohcs@gmail.com
Frode Nagel Dahl	988 67 043	fnd_89@hotmail.com
Gabriel Steinsbekk	936 59 898	gabriel.steinsbekk@universitas.no
Gard Oterholm	938 97 476	gard.oterholm@outlook.com
Geir Dorp	916 64 496	geirdo@universitas.no
Geir Molnes	993 35 518	g.molnes@gmail.com
Hallvard Østtveit Barbogen	997 69 149	hallvaob@universitas.no
Halvard Dypvik	415 25 391	halvardd@student.ilos.uio.no
Hanad Mohamed Ali	413 80 351	fl8hanad@hotmail.no
Hanna Kristin Hjardar	412 13 731	hanna@hjardar.no
Hanna Krogvold	480 25 472	hanna.magnussen@gmail.com
Hanna Skotheim	917 84 820	skotheimhanna@gmail.com
Hannah Berg	918 48 394	hannah.berg@live.no
Hanne Marie Solbø	979 68 656	hannesolbo@yahoo.no
Hans Dalane-Hval	911 94 167	hanshval@gmail.com
Hans Joseph Skjong	957 04 949	hansjskj@student.sv.uio.no
Hedda Faldet Østberg	917 51 643	hedda96@gmail.com
Heidi B Bade	476 35 276	heidibba@universitas.no
Heidi Bang		bang.heidi@gmail.com
Helene Kristiansen	934 81 673	mail@helenekristiansen.no
Heljar Havnes	928 92 479	helpel123@gmail.com
Henriette Dæhli	954 19 410	henriette_daehli@hotmail.com
Henrik Evertsson	977 75 824	evertsson.henrik@gmail.com
Henrik Giæver		henrikgiaever@hotmail.com
Hilde Mesics Kleven	952 00 026	hildemesics@gmail.com
Håken Lid	988 66 853	haakenlid@gmail.com
Håkon Benjaminsen	943 74 923	hakonben@gmail.com
Håkon Frede Foss	975 00 722	haakonfrede@gmail.com
Haakon Jamtli Kristiansen	414 50 393	haakon.jamtli@gmail.com
Håkon Sukuvara	979 81 534	h-kon@live.no
Håvard Røsæg	954 59 260	haavard24@live.com
Ida Andersen	928 67 664	idahelene.andersen@gmail.com
Ida Gøytil	981 38 985	ida_goy@hotmail.com
Ida Hove Solberg	916 05 474	idah.solberg@rocketmail.com
Ida Lyngstad Wern	958 67 095	ida.werno@gmail.com
Ida Lyngstad Wernø	958 67 095	ida.werno@gmail.com
Ida Wammer	940 33 277	ida.wammer@gmail.com
Ingeborg Huse Amundsen	922 76 929	ingeborg.ha@gmail.com
Ingeborg Misje	954 10 414	fukausen@hotmail.com
Ingri Gudmundsen Bergo	405 51 189	ingrigb@hotmail.com
Ingrid Eidsheim Daae	482 42 240	ingrided@student.sv.uio.no
Ingrid Ekeberg		ekeberg.ingrid@gmail.com
Ingrid Finess Evensmo	924 40 306	ingridevensmo@hotmail.com
Ingrid Gipling	481 05 754	i.gipling@gmail.com
Ingrid H. F. Brubaker	452 76 289	ihbrubak@universitas.no
Ingrid Nagell	981 15 169	inagell10@gmail.com
Ingvild Sagmoen	454 45 774	ingvildsagmoen@hotmail.com
Isabel Byrkjeflot Nærø	992 67 238	isabel.naero@gmail.com
Janni Kalafatis	464 48 150	jannifk@gmail.com
Jenny Gudmundsen	934 93 680	jennygu@universitas.no
Jenny Tenmann	454 49 764	je_te91@hotmail.com
Jo S. Refseth	408 68 016	jo.refseth@gmail.com
Joakim Pedersen Berg	479 01 390	joakimpb@gmail.com
Joakim Stene Preston	955 29 599	j.s.preston@universitas.no
Jonas Øren	988 67 582	jonas.oren@gmail.com
Julianne Bråten Mossing	950 83 494	julianne.mossing@hotmail,com
Julie Bjander	404 62 489	julie.bjander@hotmail.com
Julie Løvseth	907 46 147	juliebrundtland@hotmail.com
Julius V. Langhoff	922 52 407	juliusv@universitas.no
Jørgen Brynhildsvoll	980 56 586	jorgebry@universitas.no
Jørgen Kvalsvik	414 44 437	jorgenhkva@gmail.com
Kaja Skatvedt	918 01 735	kajaskatvedt@gmail.com
Kaja Storrøsten	482 56 156	kajastorrosten@gmail.com
Kari Eiring	979 81 175	karieiring@gmail.com
Karoline Tynes	932 61 149	karoline.tynes@xi.no
Katharina Simonsen	992 63 933	katharsi@universitas.no
Kathrine Salhus	911 24 753	kathrinepine@gmail.com
Katrine Myra	402 10 608	katrinmy@universitas.no
Kenneth Haug	950 16 594	kenneth.kandolf.haug@gmail.com
Kenneth Wangen	980 84 048	kenneth.wangen@outlook.com
Ketil Blom	482 78 791	ketilbh@universitas.no
Kjetil Tørum	918 36 029	kjetil.torum@gmail.com
Knut Arne Oseid	900 90 332	oseid.knut@gmail.com
Knut Ward Heimdal	980 99 488	knut.heimdal@gmail.com
Kristian Voldnes	452 53 948	kristavo@uio.no
Kristian Wikborg Wiese	992 76 956	kristian.wikborg@gmail.com
Kristina Elisabet Kvammen	962 23 623	kristinaelisabet@gmail.com
Kristina Holt	986 23 799	kristina.holt@icloud.com
Kristine Alsaker	915 49 708	kr.alsaker@gmail.com
Kristine Djuvik Kro	906 42 942	kristinekro@outlook.com
Kristoffer Hunstad	995 90 579	hunstad89@gmail.com
Lene Sørøy Neverdal	958 13 738	lene_heverdal@hotmail.com
Lina Christensen	970 97 251	linachristens@gmail.com
Line Hårklau	971 04 907	line_harklau@hotmail.com
Line Ørnes Søndergaard	920 41 849	line.sondergaard@gmail.com
Lise Blekastad	974 75 088	liseblekastad@gmail.com
Lise Grønskar	480 28 064	lise.gronskar@universitas.no
Louisa Boulaziz	400 94 261	louisaboulaziz@gmail.com
Louise Faldalen Prytz	936 02 832	l.f.prytz@universitas.no
Mads Randen	454 00 166	madsranden95@gmail.com
Magnus Godvik Ekeland	924 69 786	magnusekeland@hotmail.com
Magnus Lysberg	943 66 089	magnus.lysberg@universitas.no
Magnus Løvold	415 23 901	magnulo@universitas.no
Magnus Newth	404 70 501	mgvnewth@gmail.com
Maiken K. A. Alm	930 23 037	maiken_alm@hotmail.com
Malin Kvande	906 71 182	malinkvande@gmail.com
Maren Ørstavik	480 08 403	marenor@universitas.no
Mari Mjaaland	473 47 210	mari.mjaaland@gmail.com
Maria Mår Johansen	926 11 363	maar.johansen@googlemail.com
Maria Pettrém		mariapettrem@gmail.com
Maria Terese Kittilsen	934 94 004	mariakittilsen@gmail.com
Maria Torkilsen Horvei	480 36 708	mariahorvei@gmail.com
Marie De Rosa	412 58 848	mtrosa@universitas.no
Marit Maanum Simonsen	938 30 091	marit.simonsen@gmail.com
Markus Slettholm	993 99 054	slettholm@gmail.com
Marte Helene Mellerud	936 83 238	marte.helene.mellerud@live.no
Marthe Amanda Vannebo	480 45 872	martheva@universitas.no
Marthe Olstad	919 10 064	marolsta@gmail.com
Martin Grevstad	913 91 244	martigre@universitas.no
Martine Engebretsen Li	975 02 400	martine.engebretsen.li@gmail.com
Mathias Gravdehaug	415 97 277	mathias.gravde@gmail.com
Mathias Vedeler	415 29 011	mathiav@universitas.no
Mats Johannesen	934 06 680	mats@matsj.net
Matthis Kleeb Solheim	482 99 849	matthiskleeb@hotmail.com
Melisa Fajkovic	902 36 225	melisafajkovic@gmail.com
Mikael Lunde	905 39 870	mikaellu@universitas.no
Monica Reigstad	930 30 437	monicrei@universitas.no
Morten Oftedal Schwencke	977 73 328	morten.schwencke@gmail.com
Nathalie Wik Lystad	970 63 008	nathalielystad@gmail.com
Nicolay Woldsdal	971 43 649	nwoldsdal@gmail.com
Nora Gaupseth	922 36 291	nora.gaupseth@gmail.com
Nora Nygaard	952 82 722	noranygaard@hotmail.com
Oda Kristin Korneliussen	467 68 503	odakristin222@gmail.com
Odin Drønen	958 28 296	odindro@gmail.com
Odin Hørthe Omdal	932 43 725	odinho@universitas.no
Odin Jæger	473 98 053	odinja@gmail.com
Ole-Fredrik Lambertsen	452 59 226	olambertsen@gmail.com
Patrick da Silva Sæther	452 94 105	patrick.silva.saether@gmail.com
Paul Patrick Børhaug	926 49 598	paul_borhaug@yahoo.no
Peder Stabell	476 23 501	pederstabell@gmail.com
Peter Tryggestad	992 36 957	peter.tryggestad@gmail.com
Petter Fløttum	476 10 639	petter.flottum@gmail.com
Philip André Johannseborg	993 97 202	philip.johannesborg@gmail.com
Pål Sindre Brunstad	995 96 125	palsbrunstad@gmail.com
Rachel Antonsen		racheleantonsen@gmail.com
Ragnhild Sofie Selstø	400 42 144	ragnhild.sofie@hotmail.com
Ragnhild Torstensen	934 81 183	ragnt@universitas.no
Raisa Porsanger	415 21 576	raisa.porsanger@gmail.com
Reidar Schei Jessen	452 22 800	reidar.jessen@gmail.com
Runa Fjellanger	901 59 831	runafjellanger@gmail.com
Runar B. Mæland	975 64 932	runarbm@gmail.com
Sander Brink	942 57 159	sander@brink.no
Signe Rosenlund-Hauglid	924 49 934	s.rosenlund.hauglid@gmail.com
Sigurd Arnkleiv Bækkelund	468 02 665	sbaekkelund@yahoo.no
Sigurd Oland Nedrelid	997 84 965	s.o.nedrelid@gmail.co
Silje Kleven	908 07 861	silje.kristin.kleven@gmail.com
Siri Øverland Eriksen	936 31 040	sirioeriksen@gmail.com
Sjur Gausemel Stølen 	477 53 805	sjur@sjurgs.com
Sjur Øvrebø	990 21 608	sjur_haraldset3@hotmail.com
Skjalg Bøhmer Vold	926 11 898	skjalgen@gmail.com
Solveig Nygaard Langvad	934 86 295	solveig.langvad@gmail.com
Sondre Moen Myhre	907 46 802	sondre.myhre1@gmail.com
Stella Matia Heieren	414 13 133	stellamh@universitas.no
Stine Gudmundsen Bergo	0	stine_gb_@hotmail.com
Stine Hesstvedt	959 78 050	stinehesstvedt@gmail.com
Stine Marie Korsfur	478 09 496	stinemako@hotmail.com
Stine Stenhaug	452 61 613	stinestenh@gmail.com
Thea Bech	979 77 362	theabech@hotmail.com
Thea Marie Astrup	413 56 679	theamarieastrup@gmail.com
Thea Skyvulstad	900 73 404	theaskyvulstad@gmail.com
Thea Storøy Elnan	479 64 336	theaselnan@gmail.com
Thorbjørn Kringlebotn Borlaug	413 13 690	takborlaug@gmail.com
Tia Kristine Karlsen	930 19 878	tiakk@universitas.no
Tika Sofia León	922 19 547	tsleon90@gmail.com
Tiril Kyrkjebø	976 87 506	tirilkb@hotmail.com
Tonje Thilesen	936 00 479	tonje.thilesen@universitas.no
Torgeir Mortensen	454 72 320	torgeirm3b@gmail.com
Torill Gaarder	975 72 063	torill.gaarder@gmail.com
Vebjørn Wold	0	vebjor2910@gmail.com
Vegard Røneid Erikstad	936 95 910	vegard_erikstad@hotmail.com
Vibeke Risvold	415 16 366	vibeke_r87@hotmail.com
Vida Sundseth Brenna	922 26 144	vida.brenna@gmail.com
Vidar Bakkeli	905 87 617	vidab@universitas.no
Vilde Boberg Svineng	930 34 053	vilde.boberg@gmail.com
Vilde Iren Borse	920 68 126	vildeborse@hotmail.com
Vilde Sagstad Imeland	993 51 017	vildesimeland@gmail.com
Vincent André Placht	991 06 394	vplacht@gmail.com
Xueqi Pang	936 80 955	xueqipang@hotmail.no
Zjenja Amundsen	452 64 855	zjenjaam@outlook.com
Ørjan Ryland	934 40 276	orjan.ryland@gmail.com
Øystein Grønvold	902 75 922	oystein.gronvold@gmail.com
Øyvind Aukrust 	400 47 513	oyvind.aukrust@gmail.com
Øyvind Bosnes Engen	996 49 546	o.b.engen@universitas.no
Øyvind Gallefoss	980 03 342	ogallefoss@gmail.com
Åse Holte	414 52 584	aseholte@gmail.com
Åsmund Austenå Løvdal	934 96 019	asmundlovdal@gmail.com
'''

In [ ]:
from collections import namedtuple
Person = namedtuple('Person', 'name, phone, email')
lista = [Person(*line.split('\t')) for line in allesammen.strip().splitlines()]
lista.sort(key=lambda p: p.name)
 
for person in lista:
    contributor = Contributor.get_or_create(person.name)[0]
    contributor.email = person.email
    contributor.phone = person.phone
    contributor.verified = True
    contributor.save()
    print(f'{contributor.display_name:<30} : {person.name}')
